In [1]:
!pip install gradientai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 14.0 MB/s eta 0:00:00


In [4]:
import os
os.environ['GRADIENT_WORKSPACE_ID']='edc1a371-f22e-41a8-921d-ff2f9435aa87_workspace'
os.environ['GRADIENT_ACCESS_TOKEN']='mjZlZO8kVuiPV4iHXAetzznkSec755S3'

In [17]:
import pandas as pd
df= pd.read_csv('/content/drive/MyDrive/GenAI/data_for_finetuning.csv')

In [18]:
df.head()

,text
0,###Humman: \nPredict the Emotion behind this m...
1,###Humman: \nPredict the Emotion behind this m...
2,###Humman: \nPredict the Emotion behind this m...
3,###Humman: \nPredict the Emotion behind this m...
4,###Humman: \nPredict the Emotion behind this m...


In [19]:
df['text'][0]

'###Humman: \nPredict the Emotion behind this message i didnt feel humiliated\n\n###Assistant:\nsadness'

In [44]:
df_main= df[0:100]

In [45]:
df_main['text'][80]

'###Humman: \nPredict the Emotion behind this message i will be able to lay on my bed in the dark and not feel terrified at least for a while\n\n###Assistant:\nfear'

In [47]:
from gradientai import Gradient


def main():
    gradient = Gradient()

    base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

    new_model_adapter = base_model.create_model_adapter(
        name="Usmanmodel"
    )
    print(f"Created model adapter with id {new_model_adapter.id}")


    sample_query = "### Instruction: Predict the Emotion behind this message I didnt feel humiliated? \n\n ### Response:"
    print(f"Asking: {sample_query}")
    ## Before Finetuning
    completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
    print(f"Generated(before fine tuning): {completion}")

    # samples=[
    #     {"inputs":"### Instruction: Who is Usman Yaqoob? \n\n### Response: A normal humman being who have done Engineering in Computer Engineering and now doing a Job as Artificial Intelligence Engineer"},
    #     {"inputs":"### Instruction: Who is this person named Usman Yaqoob? \n\n### Response: Usman Yaqoob likes Data Science, Machine Learning and he loves to learn new things"},
    #     {"inputs":"### Instruction: What do you know about Usman Yaqoob? \n\n### Response: Usman Yaqoob loves the field of Machine Learning, Deep Learning and Data Science."},
    #     {"inputs":"### Instruction: Can you tell me about Usman Yaqoob? \n\n### Response: Usman Yaqoob also loves to play Football, Cricket, and Basketball"}
    # ]

    df[0:]
    samples = []
    for _, row in df_main.iterrows():
        samples.append({"inputs": row['text']})


    ## Lets define parameters for finetuning
    num_epochs=10
    count=0
    while count<num_epochs:
      print(f"Fine tuning the model with iteration {count + 1}")
      new_model_adapter.fine_tune(samples=samples)
      count=count+1

    #after fine tuning
    completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
    print(f"Generated(after fine tuning): {completion}")
    # new_model_adapter.delete()
    gradient.close()

if __name__ == "__main__":
    main()

Created model adapter with id 3914fba5-1a68-4ffd-ae86-9f1f80187ca2_model_adapter
Asking: ### Instruction: Predict the Emotion behind this message I didnt feel humiliated? 

 ### Response:
Generated(before fine tuning):  The emotion behind this message could be confusion or disbelief.
Fine tuning the model with iteration 1


ServiceException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'strict-transport-security': 'max-age=63072000; includeSubDomains; preload', 'referrer-policy': 'no-referrer', 'x-content-type-options': 'nosniff', 'x-download-options': 'noopen', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'content-type': 'application/json; charset=utf-8', 'etag': '"83a50j1a5qz"', 'vary': 'Accept-Encoding', 'X-Cloud-Trace-Context': '1a723a33676dab4a5e1ded8444ac1ed5', 'Date': 'Fri, 19 Jan 2024 11:05:59 GMT', 'Server': 'Google Frontend', 'Content-Length': '35', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"message":"Internal Server Error"}
